In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import imgaug as ia
from imgaug import augmenters as iaa
from CA1_Yolo import *
from keras.optimizers import Adam

In [2]:
img_dir = "../data/image/"
anno_dir = "../data/annotation/"

In [3]:
all_anno, labels = read_annotation_files(img_dir, anno_dir)

In [4]:
anchor_boxes = generateAnchorBoxes(all_anno, labels)

iter: 1 distances: 713.2931229342023
iter: 2 distances: 44.126853298954444
iter: 3 distances: 15.790391564481679
iter: 4 distances: 21.758992007317303
iter: 5 distances: 10.376526730124684
iter: 6 distances: 3.4537103472050266
iter: 7 distances: 5.615602790405642
iter: 8 distances: 4.285616413511261
iter: 9 distances: 2.586932431733599
iter: 10 distances: 1.8473281839441857
iter: 11 distances: 1.7749987180735414
iter: 12 distances: 2.3936999349708254
iter: 13 distances: 1.947375886090203
iter: 14 distances: 1.9614903839384787
iter: 15 distances: 1.663756393169904
iter: 16 distances: 1.635166484522978
iter: 17 distances: 1.4575251117778014


In [5]:
anchor_boxes

[26, 18, 34, 20, 34, 29, 36, 24, 44, 34, 48, 25, 56, 30, 63, 41, 75, 59]

In [6]:
train_size=0.8
train_valid_split = int(0.8 * len(all_anno))
np.random.shuffle(all_anno)
train_anno = all_anno[:train_valid_split]
valid_anno = all_anno[train_valid_split:]
max_boxes = max([len(anno['object']) for anno in (train_anno + valid_anno)])

In [7]:
len(train_anno)

63

In [18]:
train_generator = DataGenerator(
    annotations=train_anno,
    max_boxes=max_boxes,
    anchors=anchor_boxes,
    labels=labels,
    batch_size=2,
    width=416,
    height=416,
    shuffle=True,
    augment=True)

In [19]:
valid_generator = DataGenerator(
    annotations=valid_anno,
    max_boxes=max_boxes,
    anchors=anchor_boxes,
    labels=labels,
    batch_size=2,
    width=416,
    height=416,
    shuffle=True,
    augment=True)

In [20]:
train_model, infer_mode = YoloV3(
    numcls=len(labels),
    anchors=anchor_boxes,
    max_grid=[416, 416],
    batch_size=2,
    threshold=0.5,
    max_boxes=max_boxes)

In [21]:
train_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_76 (Conv2D)              (None, None, None, 3 864         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_72 (BatchNo (None, None, None, 3 128         conv2d_76[0][0]                  
__________________________________________________________________________________________________
leaky_re_lu_72 (LeakyReLU)      (None, None, None, 3 0           batch_normalization_72[0][0]     
__________________________________________________________________________________________________
zero_paddi

In [22]:
def dummy_loss(y_true, y_pred):
    return tf.sqrt(tf.reduce_sum(y_pred))

In [23]:
opt = Adam(lr=1e-4, clipnorm=0.001)
train_model.compile(loss=dummy_loss, optimizer=opt)

In [24]:
callback = create_callbacks()

D:\Anaconda3\envs\rtav\lib\site-packages\keras\callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [25]:
train_model.fit_generator(
    generator=train_generator,
    validation_data=valid_generator,
    verbose=1,
    steps_per_epoch=len(train_generator) * 3,
    epochs=300,
    callbacks=callback
)

Epoch 1/300


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: OOM when allocating tensor with shape[2,256,52,52] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node batch_normalization_85/FusedBatchNorm}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[loss_1/yolo_loss_layer_6_loss/Mean_2/_5871]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: OOM when allocating tensor with shape[2,256,52,52] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node batch_normalization_85/FusedBatchNorm}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

0 successf